# Demand-Based Pricing Optimization

### Intro.

#### Context
Have you ever wondered when the best time of year to book a hotel room is? Or the optimal length of stay in order to get the best daily rate? What if you wanted to predict whether or not a hotel was likely to receive a disproportionately high number of special requests?

This hotel booking dataset can help you explore those questions!

Content
This data set contains booking information for a city hotel and a resort hotel, and includes information such as when the booking was made, length of stay, the number of adults, children, and/or babies, and the number of available parking spaces, among other things.

All personally identifying information has been removed from the data.

#### What is Revenue Management
There is a key question:

    - What price should I charge for my available supply?
Revenue management is a business approach to balance supply and demand to maximize revenue.

Revenue managers have two strategic levers they can use to optimize revenue:

    - Capacity control: How many rooms to allocate to each segment
    - Pricing: What price to charge for each segment

If your hotel is one of the many that does not have a revenue management system, this workbook will provide you a simple model, that you can use as a baseline to making better pricing decisions.

#### Price Modeling and Optimal pricing

Looking at past, how many booking we could have got by dropping the rate or increasing the rate?

we need to make one key assumption:

    - Customers who paid a higher price would have paid a lower price if it had been available.

This key assumption aligns with the concept of consumer surplus in economics. Consumer surplus represents the difference between what consumers are willing to pay for a good or service and what they actually pay. When consumers pay a price lower than what they are willing to pay, they experience consumer surplus.

Maybe we study see historical data, but one shortfall in counting historical data is that we do not have the full range of possible prices. To get access to the full price spectrum, we will need to create a price model.

Price modeling answers the question:if I charge X , how many rooms will I sell? The price model will represent the relationship between a hotel's price and demand.

Typically fully deployed pricing model account for factors such as overall market demand, seasonality, competitors' pricing, Length of stay, lead times, and a myriad of other factors. As the goal of this workbook is for us to understand how price modeling and optimization work, we will focus only on seasonality and overall market demand driven by holidays.



### imports

In [1]:
import kagglehub
import os
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from dateutil.relativedelta import relativedelta
import datetime
import warnings
import folium
from folium.plugins import HeatMap
import plotly.express as px
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")

plt.style.use("fivethirtyeight")

%matplotlib inline


c:\Users\yezer\Documents\Projects\dynamic-pricing\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### download dataset

In [6]:
# Download latest version
path = kagglehub.dataset_download("jessemostipak/hotel-booking-demand")
print("Path to dataset files:", path)

def move_dataset_files(path: str):
    source_dir = f"{path}"
    project_root = os.path.split(os.getcwd())[0] # Assumes the script is run from the project root
    destination_dir = os.path.join(project_root, "data")

    try:
        # Create the destination directory if it doesn't exist
        os.makedirs(destination_dir, exist_ok=True)
        print(f"Ensured destination directory exists: {destination_dir}")

        # Check if the source directory exists
        if not os.path.isdir(source_dir):
            print(f"Error: Source directory not found at {source_dir}")
            return

        # List all files in the source directory
        files_to_move = [f for f in os.listdir(source_dir) if os.path.isfile(os.path.join(source_dir, f))]

        if not files_to_move:
            print(f"No files found in the source directory: {source_dir}")
            return

        print(f"Found {len(files_to_move)} files to move.")

        # Move each file
        for filename in files_to_move:
            source_file_path = os.path.join(source_dir, filename)
            destination_file_path = os.path.join(destination_dir, filename)
            
            # In case a file with the same name already exists in the destination
            if os.path.exists(destination_file_path):
                print(f"File {filename} already exists in {destination_dir}. Skipping.")
                continue
            
            shutil.move(source_file_path, destination_file_path)
            print(f"Moved: {filename} to {destination_dir}")

        print("Dataset moving process completed.")

    except Exception as e:
        print(f"An error occurred: {e}")

move_dataset_files(path)

100%|██████████| 1.25M/1.25M [00:00<00:00, 3.81MB/s]

Extracting files...
Path to dataset files: C:\Users\yezer\.cache\kagglehub\datasets\jessemostipak\hotel-booking-demand\versions\1
Ensured destination directory exists: c:\Users\yezer\Documents\Projects\dynamic-pricing\data
Found 1 files to move.
Moved: hotel_bookings.csv to c:\Users\yezer\Documents\Projects\dynamic-pricing\data
Dataset moving process completed.


### load data


In [2]:
project_root = os.path.split(os.getcwd())[0] # Assumes the script is run from the project root
data_dir = os.path.join(project_root, "data")
df = pd.read_csv(os.path.join(data_dir, "hotel_bookings.csv"))
print(df.shape)
df.head()

(119390, 32)


,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


### eda

Columns
- **hotel** (H1 = Resort Hotel (66%) or H2 = City Hotel (34%))
- **is_canceled** (1 = Canceled or 0 = Not Canceled)
- **lead_time** (Number of days that elapsed between the entering date of the booking into the PMS and the arrival date.)
- **arrival_date_year**
- **arrival_date_month**
- **arrival_date_week_number**
- **arrival_date_day_of_month**
- **stays_in_weekend_nights** (Number of weekend nights (Saturday or Sunday) the guest stayed or booked to stay at the hotel)
- **stays_in_week_nights** (Number of week nights (Monday to Friday) the guest stayed or booked to stay at the hotel)
- **adults**
- **children**
- **babies**
- **meal**: Type of meal booked. Categories are presented in standard hospitality meal packages: Undefined/SC – no meal package; BB – Bed & Breakfast; HB – Half board (breakfast and one other meal – usually dinner); FB – Full board (breakfast, lunch and dinner).
- **country**: Country of origin. Categories are represented in the ISO 3155–3:2013 format.
- **market_segment**: Market segment designation. In categories, the term “TA” means “Travel Agents” and “TO” means “Tour Operators”
- **distribution_channel**: Booking distribution channel. The term “TA” means “Travel Agents” and “TO” means “Tour Operators”
- **is_repeated_guest**: Value indicating if the booking name was from a repeated guest (1) or not (0)
- **previous_cancellations**: Number of previous bookings that were cancelled by the customer prior to the current booking.
- **previous_bookings_not_canceled**: Number of previous bookings not canceled by the customer prior to the current booking.
- **reserved_room_type**: Code of room type reserved.
- **assigned_room_type**: Code for the type of room assigned to the booking. Sometimes the assigned room type differs from the reserved.
- **booking_changes**: Number of changes/amendments made to the booking from the moment the booking was entered on the PMS until the moment of check-in or cancellation.
- **deposit_type**: Indication on if the customer made a deposit to guarantee the booking. This variable can assume three categories: No Deposit – no deposit was made; Non Refund – a deposit was made in the value of the total stay cost; Refundable – a deposit was made with a value under the total cost of stay.
- **agent**: ID of the travel agency that made the booking
- **company**: ID of the company/entity that made the booking or responsible for paying the booking.
- **days_in_waiting_list**: Number of days the booking was in the waiting list before it was confirmed to the customer.
- **customer_type**: This column describes the nature of the booking and falls into one of four categories:
     - Contract: This type of booking is associated with a pre-negotiated contract or an allotment (a block of rooms reserved for a specific client or purpose).
     - Group: This indicates the booking is part of a larger group reservation (e.g., a tour group, a wedding party).
     - Transient: This is an individual booking that is not linked to any group, contract, or any other individual booking. It's a standalone reservation.
     - Transient-party: This booking is also an individual (transient) one, but it is connected to at least one other individual (transient) booking. This might represent a small party of individuals booking separately but known to be together, without being a formal "Group" or under a "Contract".
- **adr**: Average Daily Rate as defined by dividing the sum of all lodging transactions by the total number of staying nights.
- **required_car_parking_spaces**: Number of car parking spaces required by the customer.
- **total_of_special_requests**: Number of special requests made by the customer (e.g. twin bed or high floor).
- **reservation_status**: Reservation last status, assuming one of three categories: Canceled – booking was canceled by the customer; Check-Out – customer has checked in but already departed; No-Show – customer did not check-in and did inform the hotel of the reason why.
- **reservation_status_date**: Date at which the last status was set. This variable can be used in conjunction with the ReservationStatus to.


In [3]:
df.describe()

,is_canceled,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,agent,company,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests
count,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119386.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,103050.000000,6797.000000,119390.000000,119390.000000,119390.000000,119390.000000
mean,0.370416,104.011416,2016.156554,27.165173,15.798241,0.927599,2.500302,1.856403,0.103890,0.007949,0.031912,0.087118,0.137097,0.221124,86.693382,189.266735,2.321149,101.831122,0.062518,0.571363
std,0.482918,106.863097,0.707476,13.605138,8.780829,0.998613,1.908286,0.579261,0.398561,0.097436,0.175767,0.844336,1.497437,0.652306,110.774548,131.655015,17.594721,50.535790,0.245291,0.792798
min,0.000000,0.000000,2015.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,6.000000,0.000000,-6.380000,0.000000,0.000000
25%,0.000000,18.000000,2016.000000,16.000000,8.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.000000,62.000000,0.000000,69.290000,0.000000,0.000000
50%,0.000000,69.000000,2016.000000,28.000000,16.000000,1.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,14.000000,179.000000,0.000000,94.575000,0.000000,0.000000
75%,1.000000,160.000000,2017.000000,38.000000,23.000000,2.000000,3.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,229.000000,270.000000,0.000000,126.000000,0.000000,1.000000
max,1.000000,737.000000,2017.000000,53.000000,31.000000,19.000000,50.000000,55.000000,10.000000,10.000000,1.000000,26.000000,72.000000,21.000000,535.000000,543.000000,391.000000,5400.000000,8.000000,5.000000


In [8]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  int64  
 2   lead_time                       119390 non-null  int64  
 3   arrival_date_year               119390 non-null  int64  
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        119390 non-null  int64  
 6   arrival_date_day_of_month       119390 non-null  int64  
 7   stays_in_weekend_nights         119390 non-null  int64  
 8   stays_in_week_nights            119390 non-null  int64  
 9   adults                          119390 non-null  int64  
 10  children                        119386 non-null  float64
 11  babies                          119390 non-null  int64  
 12  meal            

In [9]:
# checking for null values 
null = pd.DataFrame({'Null Values' : df.isna().sum(), 'Percentage Null Values' : (df.isna().sum()) / (df.shape[0]) * (100)})
null

,Null Values,Percentage Null Values
hotel,0,0.000000
is_canceled,0,0.000000
lead_time,0,0.000000
arrival_date_year,0,0.000000
arrival_date_month,0,0.000000
arrival_date_week_number,0,0.000000
arrival_date_day_of_month,0,0.000000
stays_in_weekend_nights,0,0.000000
stays_in_week_nights,0,0.000000
adults,0,0.000000


In [4]:
percentage_zero_children = (df['children'] == 0).sum() / len(df) * 100
print(f"Percentage of bookings with 0 children: {percentage_zero_children:.2f}%")

# we could do fill the NaN in children with 0, due to the lower value of NaNs
df.fillna({'children': 0}, inplace = True)

Percentage of bookings with 0 children: 92.80%


In [5]:
# adults, babies and children cant be zero at same time,
# so dropping the rows having all these zero at same time
print(df.shape)
filter = (df.children == 0) & (df.adults == 0) & (df.babies == 0)
df[filter].head()

(119390, 32)


,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
2224,Resort Hotel,0,1,2015,October,41,6,0,3,0,...,No Deposit,NaN,174.0,0,Transient-Party,0.0,0,0,Check-Out,2015-10-06
2409,Resort Hotel,0,0,2015,October,42,12,0,0,0,...,No Deposit,NaN,174.0,0,Transient,0.0,0,0,Check-Out,2015-10-12
3181,Resort Hotel,0,36,2015,November,47,20,1,2,0,...,No Deposit,38.0,NaN,0,Transient-Party,0.0,0,0,Check-Out,2015-11-23
3684,Resort Hotel,0,165,2015,December,53,30,1,4,0,...,No Deposit,308.0,NaN,122,Transient-Party,0.0,0,0,Check-Out,2016-01-04
3708,Resort Hotel,0,165,2015,December,53,30,2,4,0,...,No Deposit,308.0,NaN,122,Transient-Party,0.0,0,0,Check-Out,2016-01-05


In [6]:
print(df.shape)
df = df[~filter]
print(df.shape)

(119390, 32)
(119210, 32)


##### Where the customers come from?


In [7]:
country_wise_guests = df[df['is_canceled'] == 0]['country'].value_counts().reset_index()
country_wise_guests.columns = ['country', 'No of guests']
country_wise_guests

,country,No of guests
0,PRT,20977
1,GBR,9668
2,FRA,8468
3,ESP,6383
4,DEU,6067
...,...,...
160,KIR,1
161,ATF,1
162,TJK,1
163,SLE,1


In [8]:
basemap = folium.Map()
guests_map = px.choropleth(
    country_wise_guests,
    locations=country_wise_guests["country"],
    color=country_wise_guests["No of guests"],
    hover_name=country_wise_guests["country"],
)
guests_map.show()